<a href="https://colab.research.google.com/github/ghoshmoumita04/EnterpriseSearch/blob/SNInegrated/Enterprise_Knowledge_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#run this while integrating

# ServiceNow
SERVICENOW_INSTANCE = "https://<instance>.service-now.com"
SERVICENOW_USER = "api_user"
SERVICENOW_PASSWORD = "password"

# SharePoint / Azure AD
TENANT_ID = "xxxx"
CLIENT_ID = "xxxx"
CLIENT_SECRET = "xxxx"
SHAREPOINT_SITE_ID = "xxxx"
DOCUMENT_LIBRARY_ID = "xxxx"

# Model configs
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
GEN_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"

VECTOR_DB_PATH = "data/vector_db/index.faiss"
METADATA_PATH = "data/vector_db/metadata.pkl"


In [ ]:
# Model configs
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
GEN_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"

VECTOR_DB_PATH = "data/vector_db/index.faiss"
METADATA_PATH = "data/vector_db/metadata.pkl"


In [ ]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def chunk_text(text, chunk_size=700, overlap=100):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks


In [ ]:
import requests
import re

# ServiceNow credentials (moved from a conceptual 'config' or another cell)
SERVICENOW_INSTANCE = "https://<instance>.service-now.com"
SERVICENOW_USER = "api_user"
SERVICENOW_PASSWORD = "password"

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def chunk_text(text, chunk_size=700, overlap=100):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

def fetch_kb_articles():
    url = f"{SERVICENOW_INSTANCE}/api/now/table/kb_knowledge"
    params = {
        "sysparm_query": "workflow_state=published",
        "sysparm_fields": "number,short_description,text,sys_id,sys_updated_on",
        "sysparm_limit": 500
    }

    response = requests.get(url, auth=(SERVICENOW_USER, SERVICENOW_PASSWORD), params=params)
    response.raise_for_status()

    articles = []
    for rec in response.json()["result"]:
        full_text = clean_text(rec["text"])
        chunks = chunk_text(full_text)

        for c in chunks:
            articles.append({
                "content": c,
                "title": rec["short_description"],
                "source": "servicenow",
                "url": f"{SERVICENOW_INSTANCE}/kb_view.do?sys_kb_id={rec['sys_id']}",
                "updated": rec["sys_updated_on"]
            })
    return articles

In [ ]:
!pip install msal

from msal import ConfidentialClientApplication
import requests
import re

# SharePoint / Azure AD credentials (moved from a conceptual 'config' or another cell)
TENANT_ID = "xxxx"
CLIENT_ID = "xxxx"
CLIENT_SECRET = "xxxx"
SHAREPOINT_SITE_ID = "xxxx"
DOCUMENT_LIBRARY_ID = "xxxx"

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def chunk_text(text, chunk_size=700, overlap=100):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

def get_access_token():
    app = ConfidentialClientApplication(
        CLIENT_ID,
        authority=f"https://login.microsoftonline.com/{TENANT_ID}",
        client_credential=CLIENT_SECRET
    )
    token = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
    return token["access_token"]

def fetch_documents():
    token = get_access_token()
    headers = {"Authorization": f"Bearer {token}"}

    url = f"https://graph.microsoft.com/v1.0/sites/{SHAREPOINT_SITE_ID}/drives/{DOCUMENT_LIBRARY_ID}/root/children"
    files = requests.get(url, headers=headers).json()["value"]

    documents = []

    for f in files:
        if "file" not in f:
            continue

        content_url = f"https://graph.microsoft.com/v1.0/drives/{DOCUMENT_LIBRARY_ID}/items/{f['id']}/content"
        file_text = requests.get(content_url, headers=headers).text

        chunks = chunk_text(clean_text(file_text))
        for c in chunks:
            documents.append({
                "content": c,
                "title": f["name"],
                "source": "sharepoint",
                "url": f["webUrl"],
                "updated": f["lastModifiedDateTime"]
            })

    return documents

In [ ]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

model = SentenceTransformer(EMBEDDING_MODEL)

def embed_texts(texts):
    return model.encode(texts, show_progress_bar=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install faiss-cpu

import faiss
import pickle
import numpy as np

VECTOR_DB_PATH = "data/vector_db/index.faiss"
METADATA_PATH = "data/vector_db/metadata.pkl"

def build_vector_store(embeddings, metadata):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    faiss.write_index(index, VECTOR_DB_PATH)
    with open(METADATA_PATH, "wb") as f:
        pickle.dump(metadata, f)

def load_vector_store():
    index = faiss.read_index(VECTOR_DB_PATH)
    with open(METADATA_PATH, "rb") as f:
        metadata = pickle.load(f)
    return index, metadata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 66.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers peft accelerate datasets


In [ ]:
# finetune/finetune_lora.py
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from finetune.train_data import train_samples

model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

def tokenize(example):
    text = example["prompt"] + "\nAnswer: " + example["answer"]
    return tokenizer(text, truncation=True, padding="max_length", max_length=256)

dataset = Dataset.from_list(train_samples).map(tokenize)

trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=TrainingArguments(
        output_dir="data/lora",
        per_device_train_batch_size=2,
        num_train_epochs=3,
        logging_steps=1
    )
)

trainer.train()
model.save_pretrained("data/lora")
tokenizer.save_pretrained("data/lora")


In [ ]:
# Ensure all necessary libraries are installed for a self-contained cell
!pip install msal faiss-cpu

import numpy as np
import faiss
import pickle
import re # For clean_text
import requests # For fetching data
from msal import ConfidentialClientApplication # For SharePoint authentication
from sentence_transformers import SentenceTransformer
import os # To create directory for vector db

# Configuration constants from various parts of the notebook
SERVICENOW_INSTANCE = "https://<instance>.service-now.com"
SERVICENOW_USER = "api_user"
SERVICENOW_PASSWORD = "password"

TENANT_ID = "xxxx"
CLIENT_ID = "xxxx"
CLIENT_SECRET = "xxxx"
SHAREPOINT_SITE_ID = "xxxx"
DOCUMENT_LIBRARY_ID = "xxxx"

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
GEN_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"

VECTOR_DB_PATH = "data/vector_db/index.faiss"
METADATA_PATH = "data/vector_db/metadata.pkl"

# Text processing utilities (from cell vD2VhSMN6sN8)
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def chunk_text(text, chunk_size=700, overlap=100):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# ServiceNow fetching (from cell 9HWZn94q6x-0)
def fetch_kb_articles():
    url = f"{SERVICENOW_INSTANCE}/api/now/table/kb_knowledge"
    params = {
        "sysparm_query": "workflow_state=published",
        "sysparm_fields": "number,short_description,text,sys_id,sys_updated_on",
        "sysparm_limit": 500
    }

    response = requests.get(url, auth=(SERVICENOW_USER, SERVICENOW_PASSWORD), params=params)
    response.raise_for_status()

    articles = []
    for rec in response.json()["result"]:
        full_text = clean_text(rec["text"])
        chunks = chunk_text(full_text)

        for c in chunks:
            articles.append({
                "content": c,
                "title": rec["short_description"],
                "source": "servicenow",
                "url": f"{SERVICENOW_INSTANCE}/kb_view.do?sys_kb_id={rec['sys_id']}",
                "updated": rec["sys_updated_on"]
            })
    return articles

# SharePoint fetching (from cell il69r4S57EsW)
def get_access_token():
    app = ConfidentialClientApplication(
        CLIENT_ID,
        authority=f"https://login.microsoftonline.com/{TENANT_ID}",
        client_credential=CLIENT_SECRET
    )
    token = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
    return token["access_token"]

def fetch_documents():
    token = get_access_token()
    headers = {"Authorization": f"Bearer {token}"}

    url = f"https://graph.microsoft.com/v1.0/sites/{SHAREPOINT_SITE_ID}/drives/{DOCUMENT_LIBRARY_ID}/root/children"
    files = requests.get(url, headers=headers).json()["value"]

    documents = []

    for f in files:
        if "file" not in f:
            continue

        content_url = f"https://graph.microsoft.com/v1.0/drives/{DOCUMENT_LIBRARY_ID}/items/{f['id']}/content"
        file_text = requests.get(content_url, headers=headers).text

        chunks = chunk_text(clean_text(file_text))
        for c in chunks:
            documents.append({
                "content": c,
                "title": f["name"],
                "source": "sharepoint",
                "url": f["webUrl"],
                "updated": f["lastModifiedDateTime"]
            })

    return documents

# Embedder functions (from cell SLU9Tsb-7USC)
embedding_model_obj = SentenceTransformer(EMBEDDING_MODEL)
def embed_texts(texts):
    return embedding_model_obj.encode(texts, show_progress_bar=True)

# Vector store functions (from cell P5V974wZ7afp)
def build_vector_store(embeddings, metadata):
    # Ensure directory exists
    os.makedirs(os.path.dirname(VECTOR_DB_PATH), exist_ok=True)

    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    faiss.write_index(index, VECTOR_DB_PATH)
    with open(METADATA_PATH, "wb") as f:
        pickle.dump(metadata, f)

def load_vector_store():
    index = faiss.read_index(VECTOR_DB_PATH)
    with open(METADATA_PATH, "rb") as f:
        metadata = pickle.load(f)
    return index, metadata

# Main execution logic: try to load, if not found, build the vector store
try:
    index, metadata = load_vector_store()
    print("Vector store loaded successfully.")
except RuntimeError as e:
    if "No such file or directory" in str(e):
        print("Vector store not found. Building it now...")

        # Fetch data
        kb_articles = fetch_kb_articles()
        sp_documents = fetch_documents()
        all_metadata = kb_articles + sp_documents
        all_content = [item["content"] for item in all_metadata]

        # Embed and build
        embeddings = embed_texts(all_content)
        build_vector_store(embeddings, all_metadata)

        # Try loading again after building
        index, metadata = load_vector_store()
        print("Vector store built and loaded successfully.")
    else:
        raise # Re-raise other RuntimeError types

def retrieve(query, top_k=10):
    q_emb = embed_texts([query])
    distances, indices = index.search(np.array(q_emb), top_k)

    results = []
    for idx in indices[0]:
        results.append(metadata[idx])

    return results

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Vector store not found. Building it now...


ConnectionError: HTTPSConnectionPool(host='%3cinstance%3e.service-now.com', port=443): Max retries exceeded with url: /api/now/table/kb_knowledge?sysparm_query=workflow_state%3Dpublished&sysparm_fields=number%2Cshort_description%2Ctext%2Csys_id%2Csys_updated_on&sysparm_limit=500 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d2815170da0>: Failed to resolve '%3cinstance%3e.service-now.com' ([Errno -2] Name or service not known)"))

In [ ]:
from transformers import pipeline

GEN_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"

generator = pipeline(
    "text-generation",
    model=GEN_MODEL,
    torch_dtype="auto",
    device_map="auto"
)

def generate_answer(query, docs):
    context = "\n".join([d["content"] for d in docs])

    prompt = f"""
You are an enterprise knowledge assistant.
Answer the query using the context below.
Provide a concise summary.

Query: {query}
Context: {context}
"""

    output = generator(prompt, max_new_tokens=300)
    return output[0]["generated_text"]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from fastapi import FastAPI
from rag.retriever import retrieve
from rag.generator import generate_answer

app = FastAPI()

@app.get("/search")
def search(query: str):
    docs = retrieve(query)
    answer = generate_answer(query, docs)

    return {
        "query": query,
        "answer": answer,
        "results": [
            {
                "title": d["title"],
                "source": d["source"],
                "url": d["url"],
                "updated": d["updated"]
            }
            for d in docs
        ]
    }


In [ ]:
!pip install transformers peft accelerate datasets


In [ ]:
#this

# finetune/finetune_lora.py
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from finetune.train_data import train_samples

model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

def tokenize(example):
    text = example["prompt"] + "\nAnswer: " + example["answer"]
    return tokenizer(text, truncation=True, padding="max_length", max_length=256)

dataset = Dataset.from_list(train_samples).map(tokenize)

trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=TrainingArguments(
        output_dir="data/lora",
        per_device_train_batch_size=2,
        num_train_epochs=3,
        logging_steps=1
    )
)

trainer.train()
model.save_pretrained("data/lora")
tokenizer.save_pretrained("data/lora")
